In [3]:
from tracemalloc import stop
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
import schedule
import datetime
import time
import sys
# # Google增強DataFrame的渲染和導出
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

db = 'anuenews.db'
anuenewsWEB = 'https://news.cnyes.com/news/cat/tw_bull?exp=a'
# 時間
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
# newsToday = '2022-09-08'


def connect_sql():
    global conn,cursor,uniqueSql,soup

    # 連接到資料庫
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    uniqueSql = '''select * from news t'''
    res = requests.get(anuenewsWEB)
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text,"html.parser")

def cleanup_data():
    global df

    titletime = []
    idBefore = []
    id = []
    content = []

    # # 日期=================================================================================================
    times = soup.select('._67tN')
    for time in times:
        head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
        titletime.append(head)

    # 內容===================================================================================================
    titles = soup.select('._1xc2')
    # 1.去除特殊字符
    for title in titles:
        head,mid,tail = title.find('h3').text.partition(':')
        if head[0] == '「':
            head2 = head.replace('「','') # replace（代替），將字符替換成
            head3 = head2.find('、') # 在第幾位

            if head3 < 0: # 沒有、號
                head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
                head2 = head2[0]
                idBefore.append(head2)
            else: # 有、號
                head2 = head2[:head3]
                head2 = head2.split('」',1) 
                head2 = head2[0]
                idBefore.append(head2)
        else:
            idBefore.append(head)
        content.append(title.text)
    # 2.去除【月營收】
    for x in idBefore:
        x2 = x.find('月營收')
        if x2 < 0: # 沒有【月營收】
            id.append(x)
        else: # 有【月營收】
            x3 = x2 - 1
            x4 = x[:x3]
            id.append(x4)
    # ====================================================================================================
    data = {
       'date': titletime,
       'code': id,
       'content': content
    }

    df = pd.DataFrame(data)
    # 去除特殊字符 ===================================================
    for x in range(len(df['content'])):
        df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')

def insert_news():
    global filtered_dfToday

    # 今日新闻
    df_mask = df['date']==newsToday            # 日期為今天的存檔
    positions = np.flatnonzero(df_mask)
    filtered_df=df.iloc[positions]

    for index, row in filtered_df.iterrows():
        try:
            cursor.execute(
                """INSERT OR IGNORE INTO news 
                (date, code, content)
                values(?,?,?)""",
                (row['date'],
                row['code'],
                row['content']),
                )
            conn.commit()
            # 現在看來，commit()方法需要跟在增（insert）、刪（delete）、改（update）、查（select）的任何execute()語句後面。commit是把查詢語句提交到資料庫內，而不只是要向資料庫提交增、添的資料。
        except:
            pass
    
    # 確認當日新聞 ========================================================================
    dfToday = pd.read_sql(uniqueSql,conn)
    df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
    # df_mask = df3['date']=='2022-08-29'           # 日期為今天的存檔
    positionsToday = np.flatnonzero(df_maskToday)
    filtered_dfToday=dfToday.iloc[positionsToday]
    
def main():
    connect_sql()
    cleanup_data()
    insert_news()
    conn.close()
    print(filtered_dfToday)

def csv():
    conn2 = sqlite3.connect('anuenews.db')
    uniqueSql2 = '''select * from news t'''

    df2 = pd.read_sql(uniqueSql2,conn2)
    df3 = pd.DataFrame(df2, columns=['code','content','date'])

    # 今日新闻
    # newsToday = '2022-09-14'
    newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期

    df_mask = df3['date']==newsToday            # 日期為今天的存檔
    positions = np.flatnonzero(df_mask)
    filtered_df=df3.iloc[positions]

    newsToday2 = newsToday.replace("-","")
    # newsToday2 = pd.Timestamp.today().strftime("%Y%m%d")
    filtered_df.to_csv(f"csv/{newsToday2}鉅亨新聞.csv", encoding="big5hkscs",index=False)
    print(f'{newsToday2}導出成功')
    conn2.close()


schedule.every(30).minutes.until("23:50").do(main)
schedule.every().day.at("23:50").do(csv)


if __name__ == "__main__":
    while True:
        schedule.run_pending()
        time.sleep(1)

            date   code                                        content
4172  2022-09-16     宏全            宏全8月營收25.98億元年增39.33% 1—8月達171.47億元
4173  2022-09-16    慶豐富              慶豐富8月營收4.16億元年減2.94% 1—8月達36.50億元
4174  2022-09-16     台火               台火8月營收0.02億元年減93.17% 1—8月達0.70億元
4175  2022-09-16     欣泰               欣泰8月營收2.00億元年增8.08% 1—8月達17.32億元
4176  2022-09-16     盛弘              盛弘8月營收3.63億元年增21.72% 1—8月達28.55億元
4177  2022-09-16     日友               日友8月營收3.12億元年增7.95% 1—8月達25.65億元
4178  2022-09-16     陞泰                陞泰8月營收1.04億元年減7.57% 1—8月達8.57億元
4179  2022-09-16     宇智              宇智8月營收2.17億元年增77.21% 1—8月達16.72億元
4180  2022-09-16     統新               統新8月營收0.53億元年增48.96% 1—8月達4.58億元
4181  2022-09-16     立端              立端8月營收8.39億元年增45.98% 1—8月達57.85億元
4182  2022-09-16     和椿              和椿8月營收1.34億元年減22.44% 1—8月達13.38億元
4183  2022-09-16    亞弘電             亞弘電8月營收2.84億元年減13.45% 1—8月達19.26億元
4184  2022-09-16     關貿              關貿8月營收1.95億元年增13.95% 1—8月達15.04億元
4185  

# 導出CSV

In [2]:
import pandas as pd
import numpy as np
import sqlite3


conn2 = sqlite3.connect('anuenews.db')
uniqueSql2 = '''select * from news t'''

df2 = pd.read_sql(uniqueSql2,conn2)
df3 = pd.DataFrame(df2, columns=['code','content','date'])

# 今日新闻
newsToday = '2022-09-15'
# newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期

df_mask = df3['date']==newsToday            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df3.iloc[positions]

newsToday2 = newsToday.replace("-","")
# newsToday2 = pd.Timestamp.today().strftime("%Y%m%d")

filtered_df.to_csv(f"csv/{newsToday2}鉅亨新聞.csv", encoding="big5hkscs",index=False)

print(f'{newsToday2}導出成功')

20220915導出成功
